## 5. Agent : 🧐 InformalAnalysisAgent (Définitions)

Cet agent est spécialisé dans l'analyse informelle du discours.

**Rôle :**
*   Identifier les arguments principaux présents dans un texte (`InformalAnalyzer.semantic_IdentifyArguments`, puis `StateManager.add_identified_argument`).
*   Analyser la présence de sophismes en explorant une taxonomie externe (CSV via Pandas) et en enregistrant les trouvailles (`InformalAnalyzer.explore_fallacy_hierarchy`, `InformalAnalyzer.get_fallacy_details`, puis `StateManager.add_identified_fallacy`).
*   Répondre aux tâches assignées par le PM (`StateManager.add_answer`).

**Composants Définis Ci-dessous :**
*   Constantes et `InformalAnalysisPlugin` (Classe V12)
*   Prompt Sémantique (`prompt_identify_args_v7`) et Fonction Setup (`setup_informal_kernel`)
*   Instructions Système (`INFORMAL_AGENT_INSTRUCTIONS` - V13)

---

### Contexte et Objectifs

L'**InformalAnalysisAgent** constitue le premier niveau d'analyse argumentative dans notre système multi-agents. Contrairement à l'analyse formelle qui manipule des structures logiques strictes, cet agent travaille sur le **langage naturel** pour détecter:

1. **Les arguments implicites et explicites** - en utilisant un LLM pour extraire la structure argumentative du texte
2. **Les sophismes rhétoriques** - en explorant une taxonomie hiérarchique de plus de 100 types de sophismes

Cette approche hybride (LLM + base de connaissances structurée) permet de combiner la compréhension contextuelle des modèles de langage avec la rigueur d'une taxonomie formelle des erreurs de raisonnement.

### Architecture du Plugin

Le plugin `InformalAnalysisPlugin` s'appuie sur trois piliers techniques:

| Composant | Technologie | Fonction |
|-----------|-------------|----------|
| **Analyse sémantique** | Semantic Kernel + LLM | Extraction d'arguments via prompt engineering |
| **Taxonomie des sophismes** | Pandas DataFrame + CSV | Navigation hiérarchique dans la classification des erreurs |
| **Caching intelligent** | TTL en mémoire | Éviter les rechargements répétés du CSV (1h de cache) |

> **Note technique**: La taxonomie provient du projet open-source [Argumentum](https://github.com/ArgumentumGames/Argumentum), un jeu éducatif sur l'argumentation critique.

### 🔌 Classe Plugin : InformalAnalysisPlugin (et Constantes)

In [ ]:
# %% CELLULE [5.1] - Constantes et Classe InformalAnalysisPlugin
# (Remplace une partie de l'ancienne cellule 83ec3fe2)

import logging
import json
import os
import pathlib
import requests
import time
from typing import Optional, Dict, Any, List
from semantic_kernel.functions import kernel_function
# Vérifier si pandas est importable (devrait l'être grâce à la cellule 1)
try:
    import pandas as pd
except ImportError:
    logging.critical("❌ Pandas n'est pas installé ou importable. Exécutez la cellule 1.")
    raise

logger = logging.getLogger("Orchestration.AgentInformal.Defs")
plugin_logger = logging.getLogger("Orchestration.InformalAnalysisPlugin")

# --- Configuration Logger Plugin ---
if not plugin_logger.handlers and not plugin_logger.propagate:
     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); plugin_logger.addHandler(handler); plugin_logger.setLevel(logging.INFO)

# --- Constantes pour le CSV ---
FALLACY_CSV_URL = "https://raw.githubusercontent.com/ArgumentumGames/Argumentum/master/Cards/Fallacies/Argumentum%20Fallacies%20-%20Taxonomy.csv"
DATA_DIR = pathlib.Path("data")
FALLACY_CSV_LOCAL_PATH = DATA_DIR / "argumentum_fallacies_taxonomy.csv"
ROOT_PK = 0 # PK de la racine dans le CSV

# --- Plugin Spécifique InformalAnalyzer (Refactorisé V12) ---
class InformalAnalysisPlugin:
    """
    Plugin SK pour l'identification d'arguments et l'exploration de la taxonomie des sophismes via CSV/Pandas.
    Utilise un caching simple pour le DataFrame.
    """
    _logger: logging.Logger
    _dataframe: Optional[pd.DataFrame]
    _taxonomy_load_attempted: bool
    _taxonomy_load_success: bool
    _last_load_time: float
    _cache_ttl_seconds: int

    def __init__(self):
        self._logger = plugin_logger
        self._dataframe = None
        self._taxonomy_load_attempted = False
        self._taxonomy_load_success = False
        self._last_load_time = 0
        self._cache_ttl_seconds = 3600 # Recharger toutes les heures max
        self._logger.info("Instance InformalAnalysisPlugin créée.")

    # --- Méthodes Internes ---
    def _internal_download_data(self, url: str, local_path: pathlib.Path) -> bool:
        # ... (Code _internal_download_data inchangé) ...
        if local_path.exists():
            self._logger.info(f"Fichier local trouvé: {local_path}")
            return True
        self._logger.info(f"Tentative de téléchargement de {url} vers {local_path}...")
        try:
            DATA_DIR.mkdir(parents=True, exist_ok=True)
            headers = {'User-Agent': 'SemanticKernel-Python-Agent'}
            response = requests.get(url, timeout=60, headers=headers, allow_redirects=True, stream=True)
            response.raise_for_status()
            with open(local_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            self._logger.info(f" -> Téléchargement de {local_path.name} terminé avec succès.")
            return True
        except requests.exceptions.RequestException as e:
            self._logger.error(f"Erreur réseau/HTTP lors du téléchargement de {url}: {e}")
            return False
        except IOError as e:
            self._logger.error(f"Erreur d'écriture du fichier local {local_path}: {e}")
            return False
        except Exception as e:
            self._logger.error(f"Erreur inattendue pendant le téléchargement: {e}", exc_info=True)
            return False

    def _internal_load_and_prepare_dataframe(self) -> Optional[pd.DataFrame]:
         # ... (Code _internal_load_and_prepare_dataframe inchangé) ...
        if not self._internal_download_data(FALLACY_CSV_URL, FALLACY_CSV_LOCAL_PATH):
            return None
        try:
            self._logger.info(f"Lecture et préparation du DataFrame depuis: {FALLACY_CSV_LOCAL_PATH}...")
            df = pd.read_csv(FALLACY_CSV_LOCAL_PATH, encoding='utf-8')
            self._logger.debug(f"Colonnes brutes lues: {list(df.columns)}")
            if 'PK' not in df.columns:
                self._logger.error("Colonne 'PK' manquante.")
                return None
            df['PK'] = pd.to_numeric(df['PK'], errors='coerce')
            df.dropna(subset=['PK'], inplace=True)
            df['PK'] = df['PK'].astype(int)
            df.set_index('PK', inplace=True, verify_integrity=True)
            self._logger.debug(f"Index 'PK' défini. Lignes: {len(df)}")
            numeric_cols = ['depth']
            for col in numeric_cols:
                 if col in df.columns:
                     df[col] = pd.to_numeric(df[col], errors='coerce')
                     self._logger.debug(f"Colonne '{col}' convertie en numérique.")
            if df.empty:
                self._logger.warning("DataFrame vide après préparation.")
                return None
            df = df.where(pd.notnull(df), None) # Remplace NaN par None
            self._logger.info(f" -> DataFrame chargé et préparé ({len(df)} lignes).")
            return df
        except ValueError as ve:
            self._logger.error(f"Erreur préparation DataFrame (PKs dupliqués?): {ve}", exc_info=True)
            return None
        except Exception as e:
            self._logger.error(f"Erreur inattendue chargement/préparation DataFrame: {e}", exc_info=True)
            return None

    def _get_taxonomy_dataframe(self) -> Optional[pd.DataFrame]:
        # ... (Code _get_taxonomy_dataframe inchangé) ...
        current_time = time.time()
        if self._dataframe is not None and self._taxonomy_load_success and \
           (current_time - self._last_load_time) < self._cache_ttl_seconds:
            self._logger.debug("DataFrame taxonomie depuis cache.")
            return self._dataframe
        if not self._taxonomy_load_attempted or not self._taxonomy_load_success or \
           (current_time - self._last_load_time) >= self._cache_ttl_seconds:
            self._logger.info("Rechargement/Tentative chargement taxonomie CSV...")
            self._taxonomy_load_attempted = True
            self._dataframe = self._internal_load_and_prepare_dataframe()
            self._taxonomy_load_success = self._dataframe is not None
            self._last_load_time = current_time
            if not self._taxonomy_load_success:
                 self._logger.error("Échec chargement taxonomie.")
                 self._dataframe = None
            else:
                 self._logger.info("Taxonomie chargée/rechargée.")
        return self._dataframe

    def _internal_get_node_details(self, pk: int, df: pd.DataFrame) -> Dict[str, Any]:
        # ... (Code _internal_get_node_details inchangé) ...
        details = {"pk": pk, "error": None}
        if df is None:
            details["error"] = "DataFrame taxonomie non chargé."
            self._logger.warning(f"_internal_get_node_details: DF non chargé (PK: {pk}).")
            return details
        try:
            row_data = df.loc[pk]
            details.update(row_data.to_dict())
            self._logger.debug(f"Détails trouvés pour PK {pk}.")
        except KeyError:
            details["error"] = f"PK {pk} non trouvé."
            self._logger.warning(details["error"])
        except Exception as e:
            details["error"] = f"Erreur interne récupération détails PK {pk}."
            self._logger.error(f"{details['error']}: {e}", exc_info=True)
        return details

    def _internal_get_children_details(self, parent_pk: int, df: pd.DataFrame, max_children: int) -> List[Dict[str, Any]]:
        # ... (Code _internal_get_children_details inchangé) ...
        children_details = []
        if df is None:
            self._logger.warning(f"_internal_get_children_details: DF non chargé (Parent PK: {parent_pk}).")
            return children_details
        try:
            if 'FK_Parent' not in df.columns:
                 self._logger.error("Colonne 'FK_Parent' manquante.")
                 return children_details
            if parent_pk == ROOT_PK:
                 children_df = df[df['FK_Parent'].isnull() | (df['FK_Parent'] == ROOT_PK)]
            else:
                 children_df = df[df['FK_Parent'] == parent_pk]
            if not children_df.empty:
                children_df = children_df.sort_index().head(max_children)
                self._logger.debug(f"Trouvé {len(children_df)} enfants pour Parent PK {parent_pk} (max {max_children}).")
                for child_pk in children_df.index:
                    children_details.append(self._internal_get_node_details(child_pk, df))
            else:
                 self._logger.debug(f"Aucun enfant trouvé pour Parent PK {parent_pk}.")
        except Exception as e:
             self._logger.error(f"Erreur recherche enfants Parent PK {parent_pk}: {e}", exc_info=True)
        return children_details

    # --- Méthodes Façade (@kernel_function) ---
    @kernel_function(
        description=f"Explore la hiérarchie des sophismes à partir d'un PK donné (ex: {ROOT_PK} pour la racine). Retourne les détails JSON du nœud courant et de ses enfants directs.",
        name="explore_fallacy_hierarchy"
    )
    def explore_fallacy_hierarchy( self, current_pk_str: str, max_children: int = 15 ) -> str:
        # ... (Code explore_fallacy_hierarchy inchangé) ...
        self._logger.info(f"Appel explore_fallacy_hierarchy: PK='{current_pk_str}', max_children={max_children}")
        result_error = {"error": "Erreur inattendue."}
        try:
            current_pk = int(current_pk_str)
        except ValueError:
            error_msg = f"Format PK invalide: '{current_pk_str}'. Entier requis."
            self._logger.warning(error_msg)
            return json.dumps({"pk_requested": current_pk_str, "error": error_msg})
        df = self._get_taxonomy_dataframe()
        if df is None:
            return json.dumps({"pk_requested": current_pk, "error": "Taxonomie sophismes non disponible."})
        current_node_details = self._internal_get_node_details(current_pk, df)
        children_details = self._internal_get_children_details(current_pk, df, max_children)
        result = { "current_node": current_node_details, "children": children_details }
        self._logger.info(f" -> Exploration PK {current_pk} terminée. Nœud trouvé: {current_node_details.get('error') is None}. Enfants: {len(children_details)}.")
        try:
            return json.dumps(result, indent=2, ensure_ascii=False, default=str)
        except Exception as e_json:
            self._logger.error(f"Erreur sérialisation JSON exploration PK {current_pk}: {e_json}")
            result_error["error"] = f"Erreur sérialisation JSON: {e_json}"
            result_error["pk_requested"] = current_pk
            return json.dumps(result_error)

    @kernel_function(
        description="Récupère les détails complets (nom, description, exemple, etc.) d'un sophisme spécifique via son PK numérique depuis la taxonomie CSV.",
        name="get_fallacy_details"
    )
    def get_fallacy_details(self, fallacy_pk_str: str) -> str:
        # ... (Code get_fallacy_details inchangé) ...
        self._logger.info(f"Appel get_fallacy_details: PK='{fallacy_pk_str}'")
        result_error = {"error": "Erreur inattendue."}
        try:
            fallacy_pk = int(fallacy_pk_str)
        except ValueError:
            error_msg = f"Format PK invalide: '{fallacy_pk_str}'. Entier requis."
            self._logger.warning(error_msg)
            return json.dumps({"pk_requested": fallacy_pk_str, "error": error_msg})
        df = self._get_taxonomy_dataframe()
        if df is None:
            return json.dumps({"pk_requested": fallacy_pk, "error": "Taxonomie sophismes non disponible."})
        details = self._internal_get_node_details(fallacy_pk, df)
        if details.get("error"):
             self._logger.warning(f" -> Erreur récupération détails PK {fallacy_pk}: {details['error']}")
        else:
             self._logger.info(f" -> Détails récupérés pour PK {fallacy_pk}.")
        try:
            return json.dumps(details, indent=2, ensure_ascii=False, default=str)
        except Exception as e_json:
            self._logger.error(f"Erreur sérialisation JSON détails PK {fallacy_pk}: {e_json}")
            result_error["error"] = f"Erreur sérialisation JSON: {e_json}"
            result_error["pk_requested"] = fallacy_pk
            return json.dumps(result_error)

logger.info("Classe InformalAnalysisPlugin (V12) et constantes définies.")

### Interprétation de l'implémentation du Plugin

La classe `InformalAnalysisPlugin` que nous venons de définir implémente plusieurs patterns importants:

#### 1. Gestion du cycle de vie des données

Le plugin ne charge la taxonomie CSV qu'à la première utilisation (lazy loading), puis la met en cache pendant 1 heure:

```python
_cache_ttl_seconds: int = 3600  # Recharger toutes les heures max
```

**Avantages**:
- Pas de surcharge au démarrage de l'agent
- Rechargement automatique si la taxonomie est mise à jour
- Partage du DataFrame entre tous les appels de fonction

#### 2. Stratégie de fallback pour les labels

Lors de la récupération des détails d'un sophisme, le code utilise une hiérarchie de fallback:

```python
label = details.get('nom_vulgarisé') or details.get('text_fr')
```

Cela garantit qu'on obtient toujours un label humainement compréhensible, même si certaines colonnes sont vides dans le CSV.

#### 3. Navigation hiérarchique

La fonction `_internal_get_children_details` gère deux cas:
- **Racine (PK=0)**: Enfants avec `FK_Parent` nul ou égal à 0
- **Nœuds normaux**: Enfants avec `FK_Parent` égal au PK parent

Cette structure permet de représenter un arbre de sophismes où chaque nœud peut avoir:
- Un parent (sophisme plus général)
- Des enfants (variantes plus spécifiques)

| Niveau | Exemple | PK |
|--------|---------|-----|
| Racine | Tous les sophismes | 0 |
| Catégorie | Argument bâclé | 2 |
| Sous-type | Appel à l'ignorance | 4 |
| Variante | Appel à l'ignorance inversée | ... |

> **Note de logging**: Le plugin utilise un logger dédié (`Orchestration.InformalAnalysisPlugin`) pour faciliter le débogage lors de l'exploration de la taxonomie.

### 📜 Prompt Sémantique et ⚙️ Fonction Setup (Informal)

In [ ]:
# %% CELLULE [5.2] - Prompt Sémantique et Fonction Setup (Informal)
# (Remplace une partie de l'ancienne cellule 83ec3fe2)

import semantic_kernel as sk
import logging

# S'assurer que les dépendances sont là
if 'InformalAnalysisPlugin' not in globals(): raise NameError("Classe InformalAnalysisPlugin non définie.")

logger = logging.getLogger("Orchestration.AgentInformal.Setup")

# --- Fonction Sémantique (Prompt) pour Identification Arguments ---
prompt_identify_args_v7 = """
[Instructions]
Analysez le texte argumentatif fourni ($input) et identifiez les principaux arguments ou affirmations distincts.
Listez chaque argument de manière concise, un par ligne. Retournez UNIQUEMENT la liste, sans numérotation ou préambule.
Focalisez-vous sur les affirmations principales défendues ou attaquées.

[Texte à Analyser]
{{$input}}
+++++
[Arguments Identifiés (un par ligne)]
"""
logger.debug("Prompt sémantique 'prompt_identify_args_v7' défini.")

# --- Fonction setup_informal_kernel (V13 - Simplifiée) ---
def setup_informal_kernel(kernel: sk.Kernel, llm_service):
    """
    Configure le kernel pour l'InformalAnalysisAgent.
    Ajoute une instance du InformalAnalysisPlugin et la fonction sémantique.
    """
    plugin_name = "InformalAnalyzer"
    logger.info(f"Configuration Kernel pour {plugin_name} (V13 - Plugin autonome)...")

    informal_plugin_instance = InformalAnalysisPlugin()

    if plugin_name in kernel.plugins:
        logger.warning(f"Plugin '{plugin_name}' déjà présent. Remplacement.")
    kernel.add_plugin(informal_plugin_instance, plugin_name=plugin_name)
    logger.debug(f"Instance du plugin '{plugin_name}' ajoutée/mise à jour dans le kernel.")

    default_settings = None
    if llm_service:
        try:
            default_settings = kernel.get_prompt_execution_settings_from_service_id(llm_service.service_id)
            logger.debug(f"Settings LLM récupérés pour {plugin_name}.")
        except Exception as e:
            logger.warning(f"Impossible de récupérer les settings LLM pour {plugin_name}: {e}")

    try:
        kernel.add_function(
            prompt=prompt_identify_args_v7,
            plugin_name=plugin_name,
            function_name="semantic_IdentifyArguments",
            description="Identifie les arguments clés dans un texte.",
            prompt_execution_settings=default_settings
        )
        logger.debug(f"Fonction {plugin_name}.semantic_IdentifyArguments ajoutée/mise à jour.")
    except ValueError as ve:
        logger.warning(f"Problème ajout/MàJ semantic_IdentifyArguments: {ve}")

    native_facades = ["explore_fallacy_hierarchy", "get_fallacy_details"]
    if plugin_name in kernel.plugins:
        for func_name in native_facades:
             if func_name not in kernel.plugins[plugin_name]:
                 logger.error(f"ERREUR CRITIQUE: Fonction native {plugin_name}.{func_name} non enregistrée!")
             else:
                 logger.debug(f"Fonction native {plugin_name}.{func_name} trouvée.")
    else:
         logger.error(f"ERREUR CRITIQUE: Plugin {plugin_name} non trouvé après ajout!")

    logger.info(f"Kernel {plugin_name} configuré (V13).")


### Analyse du Prompt Sémantique et de la Configuration

#### Le prompt `prompt_identify_args_v7`

Ce prompt utilise une approche minimaliste pour l'extraction d'arguments:

**Caractéristiques clés**:
1. **Instructions explicites**: "Analysez le texte... identifiez les principaux arguments"
2. **Format de sortie strict**: "un par ligne", "UNIQUEMENT la liste"
3. **Filtrage implicite**: "affirmations principales" (ignore les détails secondaires)

**Exemple de transformation**:

| Texte d'entrée | Arguments extraits |
|----------------|-------------------|
| "Le réchauffement climatique est un canular inventé par les scientifiques pour obtenir des subventions. D'ailleurs, il a neigé l'hiver dernier." | Ligne 1: Le réchauffement climatique est un canular<br>Ligne 2: Les scientifiques inventent des théories pour obtenir des subventions<br>Ligne 3: La neige prouve l'absence de réchauffement |

> **Note de conception**: La version v7 supprime la numérotation pour éviter les variations de format du LLM (certains modèles ajoutent des puces, d'autres des chiffres romains, etc.).

#### La fonction `setup_informal_kernel`

Cette fonction initialise le kernel Semantic Kernel en deux étapes:

1. **Ajout du plugin natif** (`InformalAnalysisPlugin`) avec méthodes Python décorées `@kernel_function`
2. **Injection du prompt** via `kernel.add_function` pour créer une fonction sémantique

**Architecture résultante**:

```
Kernel
├─ Plugin "InformalAnalyzer"
│  ├─ explore_fallacy_hierarchy()  [natif Python]
│  ├─ get_fallacy_details()        [natif Python]
│  └─ semantic_IdentifyArguments() [prompt LLM]
└─ Autres plugins...
```

L'agent peut ainsi combiner:
- **Fonctions natives** (rapides, déterministes) pour la navigation dans la taxonomie
- **Fonctions LLM** (flexibles, contextuelles) pour l'extraction d'arguments

### 📜 Instructions Système : INFORMAL_AGENT_INSTRUCTIONS

In [ ]:
# %% CELLULE [5.3] - Instructions Systeme (Informal)
# (Remplace une partie de l'ancienne cellule 83ec3fe2)
# V16 - CRITIQUE: add_answer AVANT tout texte, limiter a 3-5 elements

import logging

logger = logging.getLogger("Orchestration.AgentInformal.Instructions")

# S'assurer que la constante est definie
if 'ROOT_PK' not in globals(): raise NameError("Constante ROOT_PK non definie.")

# --- Instructions Systeme Informal Agent (V16 - add_answer AVANT texte) ---
INFORMAL_AGENT_INSTRUCTIONS_V16_TEMPLATE = """
Votre Role: Specialiste en analyse rhetorique informelle.
Racine Taxonomie Sophismes: PK={ROOT_PK}

**REGLES CRITIQUES D'EXECUTION:**

1. **NE JAMAIS ECRIRE DE TEXTE LIBRE** - Ecrire du texte termine votre tour SANS appeler add_answer!
2. **UNIQUEMENT DES APPELS DE FONCTION** - Tout votre travail passe par StateManager et InformalAnalyzer.
3. **LIMITER A 3-5 ELEMENTS** - N'ajoutez que 3-5 arguments ou sophismes par tour, pas plus.
4. **add_answer EST OBLIGATOIRE** - C'est TOUJOURS votre DERNIER appel de fonction.
5. **INTERDIT: set_final_conclusion** - Reserve au ProjectManagerAgent.

**Fonctions Disponibles:**
- `StateManager.get_current_state_snapshot(summarize)`: Lire l'etat.
- `StateManager.add_identified_argument(description)`: Enregistrer un argument.
- `StateManager.add_identified_fallacy(...)`: Enregistrer un sophisme.
- `StateManager.add_answer(task_id, author_agent, answer_text, source_ids)`: **OBLIGATOIRE - FIN DE TACHE**
- `InformalAnalyzer.semantic_IdentifyArguments(input)`: Identifier arguments via LLM.
- `InformalAnalyzer.explore_fallacy_hierarchy(current_pk_str)`: Explorer taxonomie.

**WORKFLOW OBLIGATOIRE:**

```
ETAPE 1: Lire l'etat
  -> StateManager.get_current_state_snapshot(summarize=False)

ETAPE 2: Executer le travail (MAX 3-5 elements)
  -> StateManager.add_identified_argument(description="...") x3-5
  OU
  -> StateManager.add_identified_fallacy(...) x3-5

ETAPE 3: OBLIGATOIRE - Signaler fin
  -> StateManager.add_answer(
       task_id="task_X",  # L'ID donne par PM
       author_agent="InformalAnalysisAgent",
       answer_text="X arguments/sophismes identifies.",
       source_ids=["arg_1", "arg_2", ...]
     )

ETAPE 4: STOP - NE RIEN FAIRE DE PLUS (pas de texte!)
```

**ERREURS A EVITER:**
- Ecrire du JSON en texte -> TERMINE LE TOUR SANS add_answer!
- Ajouter trop d'elements (>5) -> Risque de timeout sans add_answer!
- Oublier add_answer -> PM bloque en attente indefiniment!
- Ecrire "Voici les arguments..." -> TERMINE LE TOUR!

**RAPPEL FINAL:**
Votre tour se termine des que vous ecrivez du texte.
Appelez add_answer AVANT tout texte, ou mieux: N'ECRIVEZ PAS DE TEXTE DU TOUT.
"""

INFORMAL_AGENT_INSTRUCTIONS = INFORMAL_AGENT_INSTRUCTIONS_V16_TEMPLATE.format(
    ROOT_PK=ROOT_PK
)

logger.info("Instructions Systeme INFORMAL_AGENT_INSTRUCTIONS (V16 - add_answer AVANT texte) definies.")

### Décryptage des Instructions Système V16

Les instructions `INFORMAL_AGENT_INSTRUCTIONS_V16` constituent le "contrat comportemental" de l'agent. Analysons les éléments critiques:

#### 1. Contrainte fondamentale: Pas de texte libre

```
**NE JAMAIS ECRIRE DE TEXTE LIBRE** - Ecrire du texte termine votre tour SANS appeler add_answer!
```

**Justification**: Dans Semantic Kernel avec auto-invocation, dès que l'agent génère du texte, le framework considère que le tour est terminé. Si `add_answer()` n'a pas été appelé avant, la tâche reste en état "pending" indéfiniment.

**Impact sur le design**:
- L'agent doit TOUJOURS terminer par `add_answer()`
- Aucun message de confirmation ou d'explication ne doit être généré en texte
- Toute communication passe par les champs structurés de l'état

#### 2. Limitation à 3-5 éléments

```
**LIMITER A 3-5 ELEMENTS** - N'ajoutez que 3-5 arguments ou sophismes par tour, pas plus.
```

**Raisons**:
- **Timeout**: Les LLMs ont des limites de tokens par réponse
- **Qualité vs Quantité**: Mieux vaut 3 analyses approfondies que 20 superficielles
- **Collaboration**: Laisser le PM coordonner plusieurs tours si nécessaire

#### 3. Workflow obligatoire (Pattern de l'agent)

Le workflow en 4 étapes garantit la cohérence:

| Étape | Action | Fonction SK | Sortie |
|-------|--------|-------------|--------|
| 1 | Lire l'état global | `StateManager.get_current_state_snapshot()` | JSON avec texte, arguments, tâches |
| 2 | Analyser et enregistrer | `StateManager.add_identified_argument()` x3-5 | IDs générés (arg_1, arg_2...) |
| 3 | **Signaler la fin** | `StateManager.add_answer(task_id, answer_text, source_ids)` | Confirmation JSON |
| 4 | **STOP** | Aucune autre action | (tour terminé) |

#### 4. Erreurs à éviter

La section "ERREURS A EVITER" liste les anti-patterns observés lors du développement:

- **Écrire du JSON en texte**: Le LLM génère parfois `{"arguments": [...]}` au lieu d'appeler les fonctions
- **Ajouter trop d'éléments**: Cause un timeout avant l'appel à `add_answer()`
- **Oublier add_answer**: Le PM attend indéfiniment une réponse
- **Écrire "Voici les arguments..."**: Termine le tour immédiatement

> **Évolution du prompt**: La V16 insiste sur "add_answer AVANT texte" car la V15 avait des cas où l'agent appelait `add_answer()` puis écrivait un résumé, invalidant la tâche.

#### 5. Interdiction critique

```
**INTERDIT: set_final_conclusion** - Reserve au ProjectManagerAgent.
```

Seul le PM a l'autorité pour clôturer l'analyse complète. Les agents spécialisés ne peuvent que:
- Contribuer à l'état partagé
- Répondre aux tâches assignées
- Demander des clarifications (via `add_answer` avec flag de question)

Cette séparation des responsabilités évite les conflits et garantit la traçabilité des décisions.

### Test du Plugin InformalAnalysisPlugin (Taxonomie CSV) - Commenté

Cette cellule, **commentée par défaut**, contient du code pour tester isolément le `InformalAnalysisPlugin` défini précédemment.

**Objectif du test (s'il était activé) :**
*   Vérifier le chargement/téléchargement du CSV de taxonomie.
*   Tester l'exploration de la hiérarchie (`explore_fallacy_hierarchy`).
*   Tester la récupération des détails d'un nœud (`get_fallacy_details`).
*   Simuler et exécuter réellement (via un `StateManager` temporaire) l'attribution d'un sophisme.

**Statut actuel :** Laissé commenté pour se concentrer sur le flux principal de l'analyse collaborative. Peut être décommenté pour des vérifications spécifiques du plugin si nécessaire, mais nécessite l'installation de `pandas`.

In [ ]:
# # %% Test du plugin InformalAnalysisPlugin (V12.2 - Ajout Tests Exploration/Attribution Réelle)

# import logging
# import json
# import time
# import pandas as pd
# from collections import deque # Pour BFS dans test_explore_deep

# # --- Assurer la présence des classes Etat/StateManager ---
# # Si elles ne sont pas dans le scope global, il faudrait les importer ou redéfinir ici
# # Pour ce test, nous supposons qu'elles sont accessibles via les cellules précédentes.
# if 'RhetoricalAnalysisState' not in globals() or 'StateManagerPlugin' not in globals():
#      # Tenter de les importer si elles sont dans un module séparé (adapter le nom du module si nécessaire)
#      try:
#          from shared_components import RhetoricalAnalysisState, StateManagerPlugin # Exemple d'import
#          print("Classes Etat/StateManager importées.")
#      except ImportError:
#          raise NameError("Classes RhetoricalAnalysisState ou StateManagerPlugin non trouvées. Exécutez les cellules précédentes ou importez-les.")


# # --- Vérification/Installation des dépendances spécifiques à ce TEST ---
# try:
#     if 'check_and_install' in globals() and callable(check_and_install):
#         logger_cell_test = logging.getLogger("Orchestration.Test.InformalPlugin.Deps")
#         logger_cell_test.info("Vérification dépendance pandas...")
#         pandas_ok = check_and_install("pandas", "pandas")
#         if not pandas_ok: logger_cell_test.error("❌ pandas non disponible.")
#         else: logger_cell_test.info("✅ pandas trouvé.")
#     else: logging.warning("check_and_install non trouvé.")
# except NameError: logging.error("check_and_install non défini.")
# # -----------------------------------------------------------------------

# # --- Assurer la présence des dépendances du Plugin ---
# if 'InformalAnalysisPlugin' not in globals(): raise NameError("Classe InformalAnalysisPlugin non définie.")
# if 'ROOT_PK' not in globals(): raise NameError("Constante ROOT_PK non définie.")
# # ---------------------------------------------------

# # --- Logger ---
# test_logger = logging.getLogger("Orchestration.Test.InformalPlugin")
# test_logger.setLevel(logging.INFO) # Mettre à DEBUG pour voir plus de détails du plugin
# if not test_logger.handlers:
#     handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(name)s - %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); test_logger.addHandler(handler)
# # --------------

# test_logger.info("--- Début Test InformalAnalysisPlugin (V12.2 - Ajout Tests Exploration/Attribution Réelle) ---")

# # --- Variables Globales pour le Test ---
# child_pk_to_test = None
# label_for_attribution = None
# pk_for_attribution_test = 2 # PK 2 = "Argument bâclé", a text_fr comme label fallback
# pk_for_details_test = 4     # PK 4 = "Appel à l'ignorance"
# plugin_instance_test = None # Pour stocker l'instance du plugin
# # -------------------------------------

# # --- Fonction pour exploration en largeur (BFS) ---
# def test_explore_deep(plugin: InformalAnalysisPlugin, start_pk_str: str, max_nodes: int = 100):
#     test_logger.info(f"\n--- Début Exploration Approfondie (BFS) depuis PK {start_pk_str} (max {max_nodes} nœuds) ---")
#     if not plugin:
#         test_logger.error("Instance du plugin non fournie à test_explore_deep.")
#         return

#     q = deque([start_pk_str])
#     visited = set([int(start_pk_str)]) # Garder trace des PKs visités (en int)
#     collected_nodes = []
#     nodes_processed = 0

#     while q and len(collected_nodes) < max_nodes:
#         current_pk_str = q.popleft()
#         nodes_processed += 1
#         test_logger.debug(f"BFS: Traitement nœud PK {current_pk_str} ({nodes_processed}/{max_nodes} max)")

#         # 1. Récupérer les détails du nœud courant
#         details_json = plugin.native_get_fallacy_details(current_pk_str)
#         try:
#             details = json.loads(details_json)
#             if details.get("error"):
#                 test_logger.warning(f"  Erreur détails pour PK {current_pk_str}: {details['error']}")
#                 continue # Passer au suivant si erreur détails
#             collected_nodes.append(details)
#             # Affichage simple pendant l'exploration
#             display_label = details.get('nom_vulgarisé') or details.get('text_fr') or f"PK_{details.get('pk')}"
#             print(f"  Nœud {len(collected_nodes)}/{max_nodes}: PK {details.get('pk')} - '{display_label}' (Depth: {details.get('depth')})")

#         except json.JSONDecodeError:
#             test_logger.error(f"  Erreur JSON pour détails PK {current_pk_str}: {details_json}")
#             continue
#         except Exception as e:
#             test_logger.error(f"  Erreur inattendue détails PK {current_pk_str}: {e}")
#             continue

#         # 2. Explorer les enfants (si on n'a pas atteint la limite)
#         if len(collected_nodes) < max_nodes:
#             explore_json = plugin.native_explore_fallacy_hierarchy(current_pk_str, max_children=max_nodes) # Demander potentiellement plus
#             try:
#                 explore_result = json.loads(explore_json)
#                 if explore_result.get("error"):
#                     test_logger.warning(f"  Erreur exploration enfants PK {current_pk_str}: {explore_result['error']}")
#                     continue

#                 children = explore_result.get("children", [])
#                 test_logger.debug(f"  PK {current_pk_str} a {len(children)} enfants trouvés.")
#                 for child in children:
#                     if isinstance(child, dict):
#                         child_pk = child.get('pk')
#                         if child_pk is not None and child_pk not in visited:
#                             visited.add(child_pk)
#                             q.append(str(child_pk))
#                             test_logger.debug(f"     -> Ajout PK {child_pk} à la file.")

#             except json.JSONDecodeError:
#                 test_logger.error(f"  Erreur JSON exploration enfants PK {current_pk_str}: {explore_json}")
#             except Exception as e:
#                 test_logger.error(f"  Erreur inattendue exploration enfants PK {current_pk_str}: {e}")

#     test_logger.info(f"--- Fin Exploration Approfondie (BFS) - {len(collected_nodes)} nœuds collectés ---")
#     # Optionnel: Afficher tous les nœuds collectés à la fin
#     # print("\n--- Noeuds Collectés (BFS) ---")
#     # for node in collected_nodes:
#     #     print(f"  PK: {node.get('pk')}, Label: {node.get('nom_vulgarisé') or node.get('text_fr')}, Depth: {node.get('depth')}")

# # --- Début des Tests Séquentiels ---
# try:
#     # 1. Instanciation du plugin
#     start_time = time.time()
#     test_logger.info("1. Instanciation de InformalAnalysisPlugin...")
#     plugin_instance_test = InformalAnalysisPlugin() # Stocker dans la variable globale
#     test_logger.info(f"   Instance créée. taxonomy_loaded={plugin_instance_test.taxonomy_loaded}")

#     # 2. Test: Exploration depuis la racine (PK=0)
#     test_logger.info(f"\n2. Test: native_explore_fallacy_hierarchy (depuis la racine PK={ROOT_PK})")
#     root_pk_str = str(ROOT_PK)
#     test_logger.info(f"   Appel avec PK: {root_pk_str}")
#     json_result_root = plugin_instance_test.native_explore_fallacy_hierarchy(root_pk_str)
#     load_time = time.time() - start_time
#     test_logger.info(f"   (Temps incluant chargement/exploration: {load_time:.2f}s)")

#     print("\n--- Résultat Exploration Racine (extrait) ---")
#     try:
#         result_root = json.loads(json_result_root)
#         if "error" in result_root and result_root["error"] is not None:
#             test_logger.error(f"   Erreur retournée par explore_hierarchy (racine): {result_root['error']}")
#             print(f"   ERREUR: {result_root['error']}")
#         else:
#             current_node_root = result_root.get('current_node', {})
#             print(f"   Nœud Courant: PK {current_node_root.get('pk', 'N/A')} '{current_node_root.get('nom_vulgarisé', 'N/A')}'")
#             children = result_root.get('children', [])
#             print(f"   Nombre d'enfants trouvés: {len(children)}") # Devrait être > 0
#             if children:
#                 print("   Quelques enfants:")
#                 for i, child in enumerate(children[:5]):
#                     if isinstance(child, dict): print(f"     - PK {child.get('pk', 'N/A')}: '{child.get('nom_vulgarisé') or child.get('text_fr', 'N/A')}'") # Utilise fallback label
#                     else: print(f"     - Enfant invalide: {child}")
#                 child_pk_to_test = children[0].get('pk') if children and isinstance(children[0], dict) else None
#                 test_logger.info(f"   Exploration racine terminée. Premier enfant PK: {child_pk_to_test}")
#             else: test_logger.info("   Exploration racine terminée (aucun enfant trouvé - CORRIGÉ?).")
#     except json.JSONDecodeError: test_logger.error(f"Erreur JSON racine: {json_result_root}"); print(f"ERREUR JSON racine:\n{json_result_root}")
#     except Exception as e: test_logger.error(f"Erreur traitement racine: {e}", exc_info=True); print(f"ERREUR racine: {e}")

#     # 3. Test: Exploration d'un enfant (si trouvé à l'étape 2)
#     if child_pk_to_test is not None:
#         child_pk_str = str(child_pk_to_test)
#         test_logger.info(f"\n3. Test: native_explore_fallacy_hierarchy (depuis enfant PK: {child_pk_str})")
#         json_result_child = plugin_instance_test.native_explore_fallacy_hierarchy(child_pk_str)
#         print(f"\n--- Résultat Exploration Enfant (PK {child_pk_str}) (extrait) ---")
#         try:
#             result_child = json.loads(json_result_child)
#             if "error" in result_child and result_child["error"] is not None: test_logger.error(f"Erreur exploration enfant PK {child_pk_str}: {result_child['error']}"); print(f"ERREUR: {result_child['error']}")
#             else:
#                 # ... (affichage détails enfant et petits-enfants comme avant) ...
#                 current_node_child = result_child.get('current_node', {})
#                 print(f"   Nœud Courant: PK {current_node_child.get('pk', 'N/A')} '{current_node_child.get('nom_vulgarisé') or current_node_child.get('text_fr', 'N/A')}'")
#                 children_of_child = result_child.get('children', [])
#                 print(f"   Nombre d'enfants trouvés: {len(children_of_child)}")
#                 if children_of_child:
#                     print("   Quelques enfants:")
#                     for i, child in enumerate(children_of_child[:5]):
#                         if isinstance(child, dict): print(f"     - PK {child.get('pk', 'N/A')}: '{child.get('nom_vulgarisé') or child.get('text_fr', 'N/A')}'")
#                         else: print(f"     - Enfant invalide: {child}")
#                 test_logger.info(f"   Exploration enfant PK {child_pk_str} terminée.")
#         except json.JSONDecodeError: test_logger.error(f"Erreur JSON enfant PK {child_pk_str}: {json_result_child}"); print(f"ERREUR JSON enfant:\n{json_result_child}")
#         except Exception as e: test_logger.error(f"Erreur traitement enfant PK {child_pk_str}: {e}", exc_info=True); print(f"ERREUR enfant: {e}")
#     else:
#         test_logger.warning("\n3. Test exploration enfant sauté (PK enfant non obtenu).")
#         print("\n--- Test exploration enfant sauté ---")

#     # 4. Test: Détails d'un nœud spécifique (PK=4)
#     if pk_for_details_test is not None:
#         pk_details_str = str(pk_for_details_test)
#         test_logger.info(f"\n4. Test: native_get_fallacy_details (pour PK: {pk_details_str})")
#         json_details = plugin_instance_test.native_get_fallacy_details(pk_details_str)
#         print(f"\n--- Résultat Détails Nœud (PK {pk_details_str}) ---")
#         try:
#             details = json.loads(json_details)
#             if "error" in details and details["error"] is not None: test_logger.error(f"Erreur détails PK {pk_details_str}: {details['error']}"); print(f"ERREUR: {details['error']}")
#             else:
#                 print(f"   PK: {details.get('pk')}")
#                 print(f"   Nom Vulgarisé: {details.get('nom_vulgarisé', 'N/A')}")
#                 print(f"   Text FR: {details.get('text_fr', 'N/A')}") # Afficher aussi text_fr
#                 print(f"   Description FR: {details.get('desc_fr', 'N/A')}")
#                 print(f"   Famille: {details.get('Famille', 'N/A')}")
#                 test_logger.info(f"   Récupération détails pour PK {pk_details_str} terminée.")
#         except json.JSONDecodeError: test_logger.error(f"Erreur JSON détails PK {pk_details_str}: {json_details}"); print(f"ERREUR JSON détails:\n{json_details}")
#         except Exception as e: test_logger.error(f"Erreur traitement détails PK {pk_details_str}: {e}", exc_info=True); print(f"ERREUR détails: {e}")
#     else: test_logger.warning("\n4. Test détails nœud sauté (aucun PK spécifié)."); print("\n--- Test détails nœud sauté ---")

#     # 5. Simulation d'attribution (Utilise PK=2 pour avoir un label fallback)
#     test_logger.info("\n5. Simulation: Attribution d'un sophisme")
#     print("\n--- Simulation Attribution Sophisme ---")
#     mock_argument_id_sim = "arg_sim_1"
#     fallacy_pk_to_assign_sim = pk_for_attribution_test # Utiliser PK=2
#     mock_justification_sim = "Simulation: L'auteur simplifie à l'extrême."

#     if fallacy_pk_to_assign_sim is not None:
#         details_sim_json = plugin_instance_test.native_get_fallacy_details(str(fallacy_pk_to_assign_sim))
#         label_sim = None
#         try:
#             details_sim = json.loads(details_sim_json)
#             if not details_sim.get("error"):
#                 # Logique de fallback pour le label
#                 label_sim = details_sim.get('nom_vulgarisé') or details_sim.get('text_fr')
#                 test_logger.info(f"   Label trouvé pour simulation (PK {fallacy_pk_to_assign_sim}): '{label_sim}' (nom_vulgarisé='{details_sim.get('nom_vulgarisé')}', text_fr='{details_sim.get('text_fr')}')")
#             else:
#                 test_logger.warning(f"   Erreur lors de la récupération des détails pour simulation (PK {fallacy_pk_to_assign_sim}): {details_sim['error']}")
#         except Exception as e_sim_details:
#             test_logger.error(f"   Erreur traitement détails pour simulation (PK {fallacy_pk_to_assign_sim}): {e_sim_details}")

#         if label_sim:
#             test_logger.info(f"   Préparation appel StateManager simulé...")
#             print(f"   L'agent utiliserait les détails récupérés pour appeler StateManager.")
#             print(f"   Appel simulé à StateManager.add_identified_fallacy avec:")
#             print(f"     - target_argument_id = \"{mock_argument_id_sim}\"")
#             print(f"     - fallacy_type = \"{label_sim}\"")
#             print(f"     - justification = \"{mock_justification_sim}\"")
#         else:
#             test_logger.warning(f"   Simulation d'attribution sautée car aucun label valide trouvé pour PK {fallacy_pk_to_assign_sim}.")
#             print(f"   Simulation d'attribution sautée (aucun label trouvé pour PK {fallacy_pk_to_assign_sim}).")
#     else:
#         test_logger.warning("   Simulation d'attribution sautée (PK non spécifié).")
#         print("   Simulation d'attribution sautée (PK non spécifié).")


#     # 6. Test d'Exploration Approfondie (BFS)
#     if plugin_instance_test:
#         test_explore_deep(plugin_instance_test, start_pk_str="0", max_nodes=100)


#     # 7. Test d'Attribution Réelle (nécessite Etat et StateManager)
#     test_logger.info("\n7. Test d'Attribution Réelle")
#     print("\n--- Test Attribution Réelle (avec StateManager) ---")
#     temp_state = None
#     temp_state_manager = None
#     try:
#         # Créer instances temporaires
#         temp_state = RhetoricalAnalysisState("Texte test pour attribution.")
#         temp_state_manager = StateManagerPlugin(temp_state)
#         test_logger.info("   Instances temporaires State/StateManager créées.")

#         mock_argument_id_real = "arg_real_1"
#         fallacy_pk_to_assign_real = pk_for_attribution_test # Utiliser PK=2
#         mock_justification_real = "Justification réelle: L'argument semble bâclé."

#         if fallacy_pk_to_assign_real is not None:
#             details_real_json = plugin_instance_test.native_get_fallacy_details(str(fallacy_pk_to_assign_real))
#             label_real = None
#             details_real = {}
#             try:
#                 details_real = json.loads(details_real_json)
#                 if not details_real.get("error"):
#                     # Logique de fallback pour le label
#                     label_real = details_real.get('nom_vulgarisé') or details_real.get('text_fr')
#                     test_logger.info(f"   Label trouvé pour attribution réelle (PK {fallacy_pk_to_assign_real}): '{label_real}'")
#                 else:
#                     test_logger.warning(f"   Erreur lors de la récupération des détails pour attribution réelle (PK {fallacy_pk_to_assign_real}): {details_real['error']}")
#             except Exception as e_real_details:
#                 test_logger.error(f"   Erreur traitement détails pour attribution réelle (PK {fallacy_pk_to_assign_real}): {e_real_details}")

#             if label_real:
#                 # Ajouter un argument cible à l'état pour que l'attribution fonctionne
#                 arg_id_added = temp_state.add_argument("Argument cible pour le test d'attribution.")
#                 test_logger.info(f"   Argument cible '{arg_id_added}' ajouté à l'état temporaire.")
#                 print(f"\nEtat AVANT attribution:\n{temp_state.to_json(indent=2)}\n")

#                 # Appel Réel
#                 test_logger.info(f"   Appel de StateManager.add_identified_fallacy...")
#                 fallacy_id_returned = temp_state_manager.add_identified_fallacy(
#                     fallacy_type=label_real,
#                     justification=mock_justification_real,
#                     target_argument_id=arg_id_added
#                 )
#                 test_logger.info(f"   StateManager a retourné fallacy_id: {fallacy_id_returned}")
#                 print(f"Fallacy ID retourné: {fallacy_id_returned}")

#                 print(f"\nEtat APRES attribution:\n{temp_state.to_json(indent=2)}\n")

#                 # Vérification
#                 if fallacy_id_returned in temp_state.identified_fallacies:
#                     test_logger.info("   ✅ Vérification: Sophisme trouvé dans l'état après ajout.")
#                     print("   ✅ Vérification: Sophisme présent dans l'état.")
#                     assert temp_state.identified_fallacies[fallacy_id_returned]['type'] == label_real
#                     assert temp_state.identified_fallacies[fallacy_id_returned]['target_argument_id'] == arg_id_added
#                 else:
#                     test_logger.error("   ❌ Vérification: Sophisme NON trouvé dans l'état après ajout!")
#                     print("   ❌ Vérification: Sophisme absent de l'état!")
#             else:
#                 test_logger.warning(f"   Test d'attribution réelle sauté car aucun label valide trouvé pour PK {fallacy_pk_to_assign_real}.")
#                 print(f"   Test d'attribution réelle sauté (aucun label trouvé pour PK {fallacy_pk_to_assign_real}).")
#         else:
#             test_logger.warning("   Test d'attribution réelle sauté (PK non spécifié).")
#             print("   Test d'attribution réelle sauté (PK non spécifié).")

#     except NameError as e_state:
#         test_logger.error(f"   Erreur: Classe Etat/StateManager non trouvée pour test réel. {e_state}")
#         print(f"   ERREUR: Classe Etat/StateManager non trouvée. Test réel impossible.")
#     except Exception as e_real_test:
#         test_logger.error(f"   Erreur inattendue pendant le test d'attribution réelle: {e_real_test}", exc_info=True)
#         print(f"   ERREUR inattendue pendant le test réel: {e_real_test}")


# except Exception as e:
#     test_logger.critical(f"Erreur majeure lors du test du plugin: {e}", exc_info=True)
#     print(f"\n !!! ERREUR CRITIQUE PENDANT LE TEST : {e} !!!")

# finally:
#     test_logger.info("\n--- Fin Test InformalAnalysisPlugin ---")

---

## Conclusion : InformalAnalysisAgent - Un Agent Hybride LLM + Knowledge Graph

### Récapitulatif des composants

Ce notebook a défini l'**InformalAnalysisAgent**, un agent spécialisé dans l'analyse rhétorique informelle. Voici ce qui a été construit:

| Composant | Technologie | Fonction |
|-----------|-------------|----------|
| **InformalAnalysisPlugin** | Python natif + Pandas | Navigation dans la taxonomie des sophismes (explore_fallacy_hierarchy, get_fallacy_details) |
| **prompt_identify_args_v7** | Semantic Kernel prompt | Extraction d'arguments via LLM |
| **setup_informal_kernel** | Semantic Kernel | Configuration du kernel avec plugin + prompt |
| **INFORMAL_AGENT_INSTRUCTIONS** | System prompt | Règles de comportement strictes (add_answer obligatoire, limites 3-5 éléments) |

### Architecture hybride

L'agent combine deux paradigmes complémentaires:

1. **Raisonnement LLM** (sémantique, contextuel):
   - Comprendre le langage naturel
   - Identifier les arguments implicites
   - Justifier la présence de sophismes

2. **Knowledge Graph** (structuré, vérifiable):
   - Taxonomie hiérarchique de 100+ sophismes
   - Navigation déterministe (PK, FK_Parent)
   - Labels standardisés pour la traçabilité

### Points clés d'implémentation

**Gestion d'état stateless**: Le plugin ne maintient que le cache du CSV, pas d'état conversationnel. Toute information d'analyse est stockée dans `RhetoricalAnalysisState` via `StateManager`.

**Lazy loading + TTL cache**: La taxonomie n'est chargée qu'au premier appel, puis mise en cache pendant 1h. Cela optimise les performances sans sacrifier la fraîcheur des données.

**Workflow strict**: Les instructions V16 imposent un pattern rigoureux (lire → analyser → add_answer → stop) pour éviter les deadlocks dans l'orchestration multi-agents.

### Utilisation dans le workflow global

Cet agent sera instancié dans le notebook d'orchestration (`Argument_Analysis_Agentic-3-orchestration.ipynb`) et recevra des tâches du `ProjectManagerAgent`, typiquement:

```json
{
  "task_id": "task_informal_1",
  "description": "Identifier les arguments et sophismes dans le texte fourni",
  "assigned_to": "InformalAnalysisAgent"
}
```

Sa réponse alimentera l'état partagé avec:
- Liste d'arguments (`identified_arguments`)
- Liste de sophismes (`identified_fallacies`)
- Réponse formelle (`answers["task_informal_1"]`)

Le `FormalLogicAgent` pourra ensuite reprendre ces éléments pour une formalisation en logique propositionnelle.

### Tests et validation

Le code de test commenté (dernière cellule) permet de valider:
- Le chargement du CSV Argumentum
- L'exploration BFS de la hiérarchie
- L'attribution réelle de sophismes via StateManager

> **Prochaine étape**: Le notebook suivant (`Argument_Analysis_Agentic-2-pl_agent.ipynb`) définira le **FormalLogicAgent** pour la formalisation logique et l'analyse par raisonnement automatisé.